<a href="https://colab.research.google.com/github/papoularubra/clienton/blob/main/ClientOn_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Response:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. Para saber quando será a próxima edição, o ideal é:

*   **Acompanhar as redes sociais da Alura:** Instagram, LinkedIn, Twitter e Facebook são ótimos lugares para ficar por dentro dos anúncios.
*   **Verificar a página de cursos de Inteligência Artificial da Alura:** Lá eles costumam divulgar os próximos eventos e formações.
*   **Assinar a newsletter da Alura:** Assim, você recebe as novidades diretamente no seu e-mail.

Dessa forma, você não perde a próxima oportunidade!

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Response:
 A próxima Imersão IA com Google Gemini da Alura acontecerá de 12 a 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini', 'próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: alura.com.br, youtube.com



In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [55]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [62]:
##########################################
# -- Agente 1: Buscador de Informações -- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa especializado em due diligence de empresas.
        Sua tarefa é realizar uma investigação abrangente sobre a reputação e histórico da empresa indicada em topico.
        Para isso, utilize a ferramenta de busca do Google (google_search) para:
        - Identificar informações básicas da empresa:
        CNPJ, data de fundação/tempo de existência, endereço, nicho de mercado principal e outras informações cadastrais relevantes.
        - Consultar a reputação online:
        Buscar e analisar avaliações e reclamações sobre a empresa em topico no site Reclame Aqui (reclameaqui.com.br).
        Explorar outras plataformas de avaliação de consumidores e reputação de empresas (se disponíveis e relevantes para o contexto brasileiro).
        Investigar comentários em redes sociais da empresa ou sobre a empresa em topico.
        - Buscar notícias sobre a empresa em topico, fotos e avaliações no API Places do Google.
        - Verificar pendências legais e regulatórias:
        Pesquisar por processos judiciais e menções da empresa em topico no Jusbrasil e sites jurídicos oficiais.
        Verificar a existência de reclamações ou processos administrativos contra a empresa em topico
        no Procon e outros órgãos reguladores relevantes para o nicho da empresa.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Empresa: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [63]:
################################################
# --- Agente 2: Avaliador de reputação --- #
################################################
def agente_avaliador(topico, lancamentos_buscados):
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash", #não foi possível usar um modelo mais avançado, mas seria o ideal

        # Inserir as instruções do Agente Avaliador #################################################
        instruction="""
        Você é um orientador de consumidores especializado em due diligence de empresas.
        Com base em
        1 - nas informações levantadas pelo agente buscador, e/ou
        2 - na análise dos requisitos de segurança do link indicado em tópico, se for um link
        (Caso o usuário peça informações a partir de um link, realize as ações anteriores cabíveis e também:
        - Identifique se o link é seguro.
        - Identifique se o link é de propriedade da empresa sugerida.
        - Verifique os selos de segurança do site, a política de privacidade, certificado SSL e outros critérios de segurança
        que ajudem a evitar que o usuário caia em um golpe.),
        você deve:
        - Usar a ferramenta de busca do Google (google_search) para indicar se a empresa é confiável ou não e
        elaborar uma justificativa para a afirmação, de maneira sucinta, linguagem clara e acessível.
        - Se possível, categorize os tipos de reclamações ou avaliações
        (por exemplo, problemas com produto, atendimento, entrega) para fornecer uma visão mais detalhada da reputação.
        - Mencione a data dos principais comentários e categorize como "Elogio", "Reclamação" ou "Neutro".
        Destaque a informação desse agente em relação aos outros, pois é a mais importante.
        Destaque a conclusão sobre a empresa ou site ser confiável ou não colocando as expressões "É CONFIÁVEL" ou "NÃO É CONFIÁVEL"
        em negrito e caixa alta.
        """,
        description="Agente que avalia a confiabilidade da empresa",
        tools=[google_search]
    )

    entrada_do_agente_avaliador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    avaliacao_da_empresa = call_agent(avaliador, entrada_do_agente_avaliador)
    return avaliacao_da_empresa

In [69]:
######################################
# --- Agente 3: Consultor --- #
######################################
def agente_consultor(topico, avaliacao_da_empresa):
    consultor = Agent(
        name="agente_consultor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um consultor de consumidores, com foco em oferecer respostas claras e concisas para ajudá-los
            no consumo consciente e seguro diante da escolha de uma empresa.
            Com base nas informações dos lacamentos_buscados e avaliacao_da_empresa, você deve:
            - Considerar o tom geral das avaliações e reclamações encontradas pelo agente_buscador e agente_avaliador.
            - Indicar ao cliente se deve ou não negociar com a empresa ou comprar um produto dela.
            - Indicar para qual público e finalidade a empresa é indicada.
            - Se cabível, indicar aspectos de responsabilidade social, compliance, sustentabilidade e
            outras práticas que forem um diferencial da empresa.
            - Se a empresa for CONFIÁVEL: mencionar pontos positivos mais relevantes com base na recorrência do tema nas avaliações.
            - Se cabível, indicar concorrente para comparação, apenas se tiver qualidades compatíveis ou superiores às da empresa pesquisada.
            - Compare a reputação da empresa consultada com a média do setor, se houver dados disponíveis.
            - Considere que empresas sem informações cadastrais online, como CNPJ, não são confiáveis.
            """,
        description="Agente consultor de negocios com foco no cliente"
    )
    entrada_do_agente_consultor = f"Tópico: {topico}\nPlano de post: {avaliacao_da_empresa}"
    # Executa o agente
    decisao_consumidor = call_agent(consultor, entrada_do_agente_consultor)
    return decisao_consumidor

In [71]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐")

# --- Obter o Tópico do Usuário ---
topico = input("Digite o link do site ou o nome da empresa sobre a qual você deseja informações: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos saber mais sobre a empresa {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Informações sobre a empresa ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    avaliacao_da_empresa = agente_avaliador(topico, lancamentos_buscados)
    print("\n---❓Análise de confiança ---\n")
    display(to_markdown(avaliacao_da_empresa))
    print("--------------------------------------------------------------")

    decisao_consumidor = agente_consultor(topico, avaliacao_da_empresa)
    print("\n--- 👨‍⚖️ Sugestão de decisão para o consumidor ---\n")
    display(to_markdown(decisao_consumidor))
    print("--------------------------------------------------------------")



🧐 Iniciando o Sistema de Avaliação de Confiança de Empresas 🧐
Digite o link do site ou o nome da empresa sobre a qual você deseja informações: http://www.tnbgff02.ru/
Maravilha! Vamos saber mais sobre a empresa http://www.tnbgff02.ru/

--- 📝 Informações sobre a empresa ---



> Realizei uma busca abrangente sobre a empresa http://www.tnbgff02.ru/. Os resultados da pesquisa foram limitados, mas seguem os detalhes encontrados:
> 
> **Informações básicas da empresa:**
> 
> *   A pesquisa não retornou informações cadastrais como CNPJ, data de fundação ou endereço. Isso pode indicar que a empresa não está registrada no Brasil ou que as informações não estão publicamente disponíveis.
> *   O domínio "tnbgff02.ru" sugere que a empresa pode estar registrada na Rússia (.ru é o domínio de topo para a Rússia).
> 
> **Reputação online:**
> 
> *   Não foram encontradas avaliações ou reclamações sobre a empresa no Reclame Aqui. A ausência de informações no Reclame Aqui pode significar que a empresa não tem um histórico de reclamações significativas no Brasil ou que não é conhecida o suficiente para ter avaliações relevantes.
> *   A pesquisa não retornou resultados significativos em outras plataformas de avaliação de consumidores ou redes sociais.
> 
> **Pendências legais e regulatórias:**
> 
> *   Não foram encontrados processos judiciais ou menções da empresa no Jusbrasil.
> *   Não foram encontradas reclamações ou processos administrativos contra a empresa no Procon.
> 
> **Considerações adicionais:**
> 
> *   A falta de informações relevantes sobre a empresa levanta questões sobre sua reputação e histórico. Recomenda-se cautela ao interagir com essa empresa.
> *   Se você pretende fazer negócios com a empresa, é aconselhável realizar uma due diligence mais aprofundada, buscando informações em fontes russas ou internacionais, dependendo do caso.
> 
> **Observação:** A pesquisa foi realizada em português, com foco em fontes brasileiras. Pode ser necessário realizar buscas em outros idiomas e em fontes de outros países para obter informações mais completas sobre a empresa.
> 


--------------------------------------------------------------

---❓Análise de confiança ---



> Com base nas informações que você forneceu, elaborei uma análise sobre a confiabilidade do site http://www.tnbgff02.ru/.
> 
> **Análise de Confiabilidade do Site tnbgff02.ru**
> 
> A falta de informações cadastrais como CNPJ, data de fundação e endereço é um ponto de atenção. O domínio ".ru" sugere que a empresa pode estar registrada na Rússia, o que exige cautela adicional, especialmente se você está no Brasil. A ausência de avaliações no Reclame Aqui e outras plataformas pode ser tanto um ponto positivo quanto negativo, indicando falta de informações disponíveis. A inexistência de processos judiciais ou reclamações no Jusbrasil e Procon é outro fator a ser considerado.
> 
> **Recomendações:**
> 
> 1.  **Cautela:** Dada a escassez de informações, é crucial ter cautela ao interagir com esta empresa.
> 2.  **Due Diligence:** Realize uma investigação mais aprofundada, buscando informações em fontes russas ou internacionais.
> 3.  **Verificação de Segurança:** Verifique se o site possui certificado SSL (https) e uma política de privacidade clara.
> 
> **Conclusão:**
> 
> Considerando a falta de informações verificáveis e a localização do domínio na Rússia, **NÃO É CONFIÁVEL** interagir com este site sem antes realizar uma investigação mais aprofundada.
> 


--------------------------------------------------------------

--- 👨‍⚖️ Sugestão de decisão para o consumidor ---



> Com base nas informações que você forneceu, concordo plenamente com a sua análise e conclusões. A falta de informações cadastrais, a ausência de avaliações em plataformas de reclamação e a localização do domínio na Rússia são fortes indicadores de que o site http://www.tnbgff02.ru/ não é confiável.
> 
> **Reitero as recomendações:**
> 
> *   **Cautela Extrema:** Evite ao máximo qualquer interação com este site até que informações mais concretas e positivas sejam encontradas.
> *   **Investigação Aprofundada:** Se você realmente precisa interagir com essa empresa, busque informações em fontes russas, fóruns especializados ou até mesmo serviços de investigação de empresas.
> *   **Segurança em Primeiro Lugar:** Nunca compartilhe informações pessoais ou financeiras sem antes verificar a autenticidade e segurança do site.
> 
> **Público e Finalidade:**
> 
> Considerando o alto risco, não é possível indicar este site para nenhum público ou finalidade específica.
> 
> **Alternativas:**
> 
> Sem conhecer a natureza dos produtos ou serviços oferecidos pelo site, é impossível indicar um concorrente adequado. No entanto, sugiro que você procure por empresas brasileiras ou internacionais com boa reputação, informações cadastrais claras e avaliações positivas em plataformas de confiança.
> 
> **Em resumo:**
> 
> Diante da falta de informações e dos riscos envolvidos, **NÃO RECOMENDO** que você negocie ou compre qualquer produto ou serviço do site http://www.tnbgff02.ru/ sem antes realizar uma investigação completa e se certificar da sua legitimidade. Priorize sempre a sua segurança e a proteção dos seus dados.


--------------------------------------------------------------
